In [1]:
import glob
import pandas as pd
catf = glob.glob("train/train/cat.*.jpg")
catans = [0] * len(catf)
dogf = glob.glob("train/train/dog.*.jpg")
dogans = [1] * len(dogf)
df = pd.DataFrame({
    "path":catf + dogf,
    "ans":catans + dogans
})
df

,path,ans
0,train/train\cat.0.jpg,0
1,train/train\cat.1.jpg,0
2,train/train\cat.10.jpg,0
3,train/train\cat.100.jpg,0
4,train/train\cat.1000.jpg,0
...,...,...
24995,train/train\dog.9995.jpg,1
24996,train/train\dog.9996.jpg,1
24997,train/train\dog.9997.jpg,1
24998,train/train\dog.9998.jpg,1


In [2]:
from keras.applications.vgg16 import VGG16
vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
vgg.summary()

Using TensorFlow backend.
C:\Users\User\PycharmProjects\MLDemo\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\PycharmProjects\MLDemo\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\PycharmProjects\MLDemo\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\PycharmProjects

C:\Users\User\PycharmProjects\MLDemo\venv\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\PycharmProjects\MLDemo\venv\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\PycharmProjects\MLDemo\venv\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\PycharmProjects\MLDe



Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [3]:
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
for l in vgg.layers:
    l.trainable = False
x = Flatten()(vgg.output)
x = Dense(4096, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(2, activation="softmax")(x)
cnn = Model(inputs=vgg.input, outputs=x)
cnn.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
__________

In [4]:
# 進到模型處理: numpy array
import numpy as np
x = np.array(df["path"])
y = np.array(pd.get_dummies(df["ans"]))

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,
                                                    y, 
                                                    test_size=0.1)

In [6]:
# only Demo: 藉由一張圖片告訴你如何處理
from keras.preprocessing.image import load_img
# 一定使用他當初的做法
# https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py
from keras.applications.vgg16 import preprocess_input
img = load_img(x_train[0], target_size=(224, 224)).convert("RGB")
img_np = np.array(img) 
preprocess_input(img_np)

array([[[-22.939003 , -16.779    , -42.68     ],
        [-19.939003 , -13.778999 , -39.68     ],
        [-18.939003 , -12.778999 , -38.68     ],
        ...,
        [ -8.939003 ,  -3.7789993, -33.68     ],
        [ -5.939003 ,  -6.7789993, -33.68     ],
        [ -8.939003 ,  -9.778999 , -36.68     ]],

       [[-24.939003 , -18.779    , -44.68     ],
        [-21.939003 , -15.778999 , -41.68     ],
        [-20.939003 , -14.778999 , -40.68     ],
        ...,
        [ -9.939003 ,  -4.7789993, -34.68     ],
        [ -9.939003 ,  -9.778999 , -38.68     ],
        [-12.939003 , -12.778999 , -41.68     ]],

       [[-25.939003 , -19.779    , -45.68     ],
        [-22.939003 , -16.779    , -42.68     ],
        [-21.939003 , -15.778999 , -41.68     ],
        ...,
        [ -9.939003 ,  -4.7789993, -34.68     ],
        [-11.939003 ,  -8.778999 , -38.68     ],
        [-12.939003 , -12.778999 , -41.68     ]],

       ...,

       [[-43.939003 , -34.779    , -69.68     ],
        [-4

In [7]:
# return (原圖, 預處理過圖, 答案)
def get_images(batch, x, y):
    idx = np.random.randint(0, x.shape[0], batch)
    ori_img_list = []
    img_list = []
    for fp in x_train[idx]:
        img = load_img(fp, target_size=(224, 224)).convert("RGB")
        img_np = np.array(img) 
        img_pre = preprocess_input(img_np)
        img_list.append(img_pre)
        ori_img_list.append(img_np)
    img_list_np = np.array(img_list)
    ori_img_list_np = np.array(ori_img_list)
    return (ori_img_list_np, img_list_np, y[idx])

In [11]:
cnn.compile(loss="categorical_crossentropy",
            optimizer="adam",
            metrics=["accuracy"])

In [16]:
times = 5
for i in range(times):
    imgs = get_images(20, x_train, y_train)
    result = cnn.train_on_batch(imgs[1], imgs[2])
    print("-" * 15, "times", i + 1, "-" * 15)
    print("[Train]:", result)
    # 如果時間充裕, 最好多一點
    test_imgs = get_images(20, x_test, y_test)
    test_result = cnn.test_on_batch(test_imgs[1], test_imgs[2])
    print("[Validate]:", test_result)

--------------- times 1 ---------------
[Train]: [7.2531433, 0.55]
[Validate]: [10.476763, 0.35]
--------------- times 2 ---------------
[Train]: [8.864953, 0.45]
[Validate]: [8.059048, 0.5]
--------------- times 3 ---------------
[Train]: [12.088572, 0.25]
[Validate]: [7.2531433, 0.55]
--------------- times 4 ---------------
[Train]: [8.864952, 0.45]
[Validate]: [6.447238, 0.6]
--------------- times 5 ---------------
[Train]: [5.6413336, 0.65]
[Validate]: [9.670857, 0.4]


In [17]:
cnn.predict(test_imgs[1])

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)